# Mojo 0.6
This is a test notebook that explores some of the new features in [Mojo 0.6 release](https://docs.modular.com/mojo/changelog.html?utm_content=273987722&utm_medium=social&utm_source=twitter&hss_channel=tw-1483918307484848132)

In [29]:
@value
struct test: 
  fn sayhello(self):
    print("Hello")

let t = test()
t.sayhello()

Hello


## Movable, Copyable

New traits added for move and copy constructors that are required for types that can be stored in a DynamicVector.

In [30]:

struct van(Movable, Copyable, CollectionElement):
  var thing: Int

  fn __init__(inout self):
    print("van init")
    self.thing = 0

  fn __moveinit__(inout self: van, owned existing: van ):
    print("van moveinit")
    self.thing = existing.thing

  fn __copyinit__(inout self: van, existing: van ):
    print("van copyinit")
    self.thing = existing.thing

  fn __del__(owned self):
    print("del")

  
fn testMovableCopyable():
   print("testMovableCopyable")
   var vans: DynamicVector[van] = DynamicVector[van]() 
   vans.append(van())
   print("len vans",len(vans))
   print("Done")
   # surprising del before
   # explict calling del on vans weird also

## Partial automatic parametarization

Readme says: "when a function is declared with an argument of a partially bound type, the unbound parameters of that type are implicitly added to the function’s input parameters."

This assumes knowledge of what a partially bound type is.  What is that?

From the documentation: [type bindings](https://docs.modular.com/mojo/manual/parameters/#fully-bound-partially-bound-and-unbound-types)

In [31]:
struct diary[storetype: CollectionElement]:
   var storage: DynamicVector[storetype]

   fn __init__(inout self):
      self.storage = DynamicVector[storetype]()

# when we use the diary type, we supply / bind a parameter
var recorder: diary[String] = diary[String]()

# function has a unbound form of diary as function argument
fn partiallyboundfunction(thediary: diary):
   pass

# calling funciton performas the parameter binding
partiallyboundfunction(diary[String]())

## Explicit unbound parameters

Referencing a parameterized type in a function call supplying `_` as a parameter makes it possible to disambiguate which parameters are supplied as arguments by the caller: 

The readme implies that if values are supplied for some parameters with others explicitly unbound via _, only the unbound ones need to be provided when calling.  This doesn't seem to work in the following test.

In [32]:
struct diary[storetype: CollectionElement,maxlength: Int, maxentries: Int]:
   var storage: DynamicVector[storetype]

   fn __init__(inout self):
      self.storage = DynamicVector[storetype]()

fn explicitlyunbound(thediary: diary[String, _, 10]):
    pass

explicitlyunbound(diary[1])

error: Expression [32]:10:25: 'diary' parameter #0 has 'CollectionElement' type, but value has type 'IntLiteral'
explicitlyunbound(diary[1])
                        ^

Expression [32]:1:1: 'diary' declared here
struct diary[storetype: CollectionElement,maxlength: Int, maxentries: Int]:
^

expression failed to parse (no further compiler diagnostics)

## __mlir_op supports operations that return multiple Items

This is quite advanced and requires further investigation

In [33]:
var a: Tuple[String, Int, Int] = Tuple(String("Hello"), 10, 10)

struct multiresult:

   fn test(self):
        # The `ret` variable has type `Tuple[Int, Int]`.
        let ret = __mlir_op.`multi_result_op`[ _type = (Int, Int) ]()


error: Expression [33]:7:68: use of unregistered MLIR operation 'multi_result_op'
        let ret = __mlir_op.`multi_result_op`[ _type = (Int, Int) ]()
                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^~

expression failed to parse (no further compiler diagnostics)

## Reading raw bytes from a file
This just works even in a mojo notebook

In [36]:
with open("LICENSE", "r") as f:
    data = f.read_bytes()
    print("Bytes:", data.bytecount())

Bytes: 11357
